In [1]:
from __future__ import print_function, division

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        X_train_main = np.load('X_mnist.npy')
        X_train = np.vstack([X_train_main,np.load('aug_x.npy')])
        X_train = np.reshape(X_train, (X_train.shape[0], 28, 28))

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=8001, batch_size=32, sample_interval=200)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

C:\Users\Duicane\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.633473, acc.: 48.44%] [G loss: 0.400572]
1 [D loss: 0.690754, acc.: 51.56%] [G loss: 0.433578]
2 [D loss: 0.579044, acc.: 54.69%] [G loss: 0.531880]
3 [D loss: 0.415651, acc.: 67.19%] [G loss: 0.713103]
4 [D loss: 0.340519, acc.: 87.50%] [G loss: 0.919158]
5 [D loss: 0.284415, acc.: 90.62%] [G loss: 1.028640]
6 [D loss: 0.230891, acc.: 98.44%] [G loss: 1.194134]
7 [D loss: 0.230893, acc.: 95.31%] [G loss: 1.227667]
8 [D loss: 0.194689, acc.: 96.88%] [G loss: 1.402177]
9 [D loss: 0.157791, acc.: 100.00%] [G loss: 1.480332]
10 [D loss: 0.158980, acc.: 100.00%] [G loss: 1.577672]
11 [D loss: 0.136285, acc.: 100.00%] [G loss: 1.606616]
12 [D loss: 0.138465, acc.: 100.00%] [G loss: 1.716991]
13 [D loss: 0.118263, acc.: 100.00%] [G loss: 1.859285]
14 [D loss: 0.099149, acc.: 100.00%] [G loss: 1.944941]
15 [D loss: 0.087770, acc.: 100.00%] [G loss: 2.025913]
16 [D loss: 0.086976, acc.: 100.00%] [G loss: 2.074400]
17 [D loss: 0.074923, acc.: 100.00%] [G loss: 2.182822]
18 [D loss:

157 [D loss: 0.167104, acc.: 95.31%] [G loss: 3.511166]
158 [D loss: 0.144864, acc.: 98.44%] [G loss: 3.396252]
159 [D loss: 0.222712, acc.: 87.50%] [G loss: 3.520636]
160 [D loss: 0.240744, acc.: 92.19%] [G loss: 2.694867]
161 [D loss: 0.224828, acc.: 92.19%] [G loss: 2.870665]
162 [D loss: 0.172960, acc.: 93.75%] [G loss: 3.119548]
163 [D loss: 0.105414, acc.: 96.88%] [G loss: 3.109174]
164 [D loss: 0.148668, acc.: 95.31%] [G loss: 3.156545]
165 [D loss: 0.384577, acc.: 82.81%] [G loss: 3.538654]
166 [D loss: 0.677492, acc.: 60.94%] [G loss: 3.555990]
167 [D loss: 0.166631, acc.: 95.31%] [G loss: 4.034736]
168 [D loss: 0.128701, acc.: 95.31%] [G loss: 3.043901]
169 [D loss: 0.230574, acc.: 90.62%] [G loss: 3.222741]
170 [D loss: 0.111231, acc.: 98.44%] [G loss: 3.653280]
171 [D loss: 0.531359, acc.: 79.69%] [G loss: 2.955241]
172 [D loss: 0.244792, acc.: 87.50%] [G loss: 4.132317]
173 [D loss: 0.220898, acc.: 95.31%] [G loss: 3.589592]
174 [D loss: 0.325474, acc.: 85.94%] [G loss: 2.

...

7401 [D loss: 0.567460, acc.: 71.88%] [G loss: 1.007717]
7402 [D loss: 0.606908, acc.: 71.88%] [G loss: 1.082744]
7403 [D loss: 0.645853, acc.: 56.25%] [G loss: 0.890118]
7404 [D loss: 0.673884, acc.: 59.38%] [G loss: 0.989718]
7405 [D loss: 0.615045, acc.: 76.56%] [G loss: 0.956511]
7406 [D loss: 0.599597, acc.: 62.50%] [G loss: 0.924928]
7407 [D loss: 0.566841, acc.: 65.62%] [G loss: 0.964529]
7408 [D loss: 0.615412, acc.: 71.88%] [G loss: 1.062337]
7409 [D loss: 0.553415, acc.: 81.25%] [G loss: 0.943382]
7410 [D loss: 0.585006, acc.: 73.44%] [G loss: 1.104987]
7411 [D loss: 0.596369, acc.: 68.75%] [G loss: 1.018827]
7412 [D loss: 0.626518, acc.: 70.31%] [G loss: 1.008447]
7413 [D loss: 0.634325, acc.: 64.06%] [G loss: 1.000197]
7414 [D loss: 0.646314, acc.: 57.81%] [G loss: 1.117159]
7415 [D loss: 0.604886, acc.: 68.75%] [G loss: 0.897304]
7416 [D loss: 0.590792, acc.: 71.88%] [G loss: 0.928601]
7417 [D loss: 0.536668, acc.: 73.44%] [G loss: 0.887994]
7418 [D loss: 0.627291, acc.: 5

7550 [D loss: 0.604404, acc.: 73.44%] [G loss: 1.009738]
7551 [D loss: 0.607831, acc.: 67.19%] [G loss: 0.896911]
7552 [D loss: 0.626029, acc.: 65.62%] [G loss: 1.024410]
7553 [D loss: 0.646234, acc.: 60.94%] [G loss: 1.008540]
7554 [D loss: 0.609630, acc.: 71.88%] [G loss: 0.930086]
7555 [D loss: 0.580215, acc.: 70.31%] [G loss: 0.996261]
7556 [D loss: 0.659765, acc.: 62.50%] [G loss: 1.012455]
7557 [D loss: 0.605939, acc.: 70.31%] [G loss: 1.033786]
7558 [D loss: 0.605799, acc.: 67.19%] [G loss: 0.943040]
7559 [D loss: 0.599929, acc.: 68.75%] [G loss: 0.963720]
7560 [D loss: 0.549173, acc.: 75.00%] [G loss: 0.989438]
7561 [D loss: 0.602672, acc.: 65.62%] [G loss: 0.953271]
7562 [D loss: 0.689962, acc.: 56.25%] [G loss: 0.992099]
7563 [D loss: 0.668844, acc.: 56.25%] [G loss: 1.059076]
7564 [D loss: 0.672455, acc.: 64.06%] [G loss: 1.057042]
7565 [D loss: 0.646373, acc.: 59.38%] [G loss: 1.138092]
7566 [D loss: 0.617455, acc.: 73.44%] [G loss: 1.069105]
7567 [D loss: 0.652216, acc.: 6

7696 [D loss: 0.590528, acc.: 70.31%] [G loss: 0.921639]
7697 [D loss: 0.629074, acc.: 68.75%] [G loss: 1.138933]
7698 [D loss: 0.602093, acc.: 64.06%] [G loss: 0.996249]
7699 [D loss: 0.572403, acc.: 65.62%] [G loss: 1.072652]
7700 [D loss: 0.636023, acc.: 67.19%] [G loss: 0.976131]
7701 [D loss: 0.669089, acc.: 60.94%] [G loss: 1.034151]
7702 [D loss: 0.674569, acc.: 59.38%] [G loss: 0.909599]
7703 [D loss: 0.659120, acc.: 59.38%] [G loss: 0.999214]
7704 [D loss: 0.614197, acc.: 68.75%] [G loss: 0.926872]
7705 [D loss: 0.635896, acc.: 59.38%] [G loss: 1.011920]
7706 [D loss: 0.637150, acc.: 64.06%] [G loss: 0.931312]
7707 [D loss: 0.533583, acc.: 76.56%] [G loss: 0.908683]
7708 [D loss: 0.644014, acc.: 60.94%] [G loss: 1.032499]
7709 [D loss: 0.537280, acc.: 79.69%] [G loss: 1.016018]
7710 [D loss: 0.539806, acc.: 75.00%] [G loss: 1.100768]
7711 [D loss: 0.642837, acc.: 62.50%] [G loss: 1.041988]
7712 [D loss: 0.681691, acc.: 56.25%] [G loss: 0.876034]
7713 [D loss: 0.566711, acc.: 7

7842 [D loss: 0.692722, acc.: 56.25%] [G loss: 0.961367]
7843 [D loss: 0.612956, acc.: 64.06%] [G loss: 1.038623]
7844 [D loss: 0.656088, acc.: 64.06%] [G loss: 1.024408]
7845 [D loss: 0.618412, acc.: 65.62%] [G loss: 1.042565]
7846 [D loss: 0.583372, acc.: 68.75%] [G loss: 0.967449]
7847 [D loss: 0.597406, acc.: 68.75%] [G loss: 0.879655]
7848 [D loss: 0.596591, acc.: 67.19%] [G loss: 0.962157]
7849 [D loss: 0.590712, acc.: 70.31%] [G loss: 0.928600]
7850 [D loss: 0.563050, acc.: 75.00%] [G loss: 0.987331]
7851 [D loss: 0.675011, acc.: 56.25%] [G loss: 0.958737]
7852 [D loss: 0.607341, acc.: 60.94%] [G loss: 1.039926]
7853 [D loss: 0.604178, acc.: 68.75%] [G loss: 1.159429]
7854 [D loss: 0.578943, acc.: 71.88%] [G loss: 1.091843]
7855 [D loss: 0.651622, acc.: 59.38%] [G loss: 1.011512]
7856 [D loss: 0.646368, acc.: 60.94%] [G loss: 0.973758]
7857 [D loss: 0.659906, acc.: 64.06%] [G loss: 1.057034]
7858 [D loss: 0.555463, acc.: 70.31%] [G loss: 1.020445]
7859 [D loss: 0.677665, acc.: 5

7989 [D loss: 0.590138, acc.: 70.31%] [G loss: 0.953382]
7990 [D loss: 0.581447, acc.: 71.88%] [G loss: 1.027531]
7991 [D loss: 0.562060, acc.: 70.31%] [G loss: 0.998246]
7992 [D loss: 0.607542, acc.: 71.88%] [G loss: 0.977159]
7993 [D loss: 0.573094, acc.: 65.62%] [G loss: 1.024277]
7994 [D loss: 0.582225, acc.: 65.62%] [G loss: 1.077976]
7995 [D loss: 0.583221, acc.: 70.31%] [G loss: 1.025203]
7996 [D loss: 0.598350, acc.: 76.56%] [G loss: 1.148455]
7997 [D loss: 0.618127, acc.: 65.62%] [G loss: 1.063615]
7998 [D loss: 0.640377, acc.: 59.38%] [G loss: 1.047193]
7999 [D loss: 0.562113, acc.: 75.00%] [G loss: 1.156423]
8000 [D loss: 0.647300, acc.: 65.62%] [G loss: 0.957331]
